In [ ]:
%matplotlib widget
import matplotlib

import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("./underactuated/src")

import math

from underactuated.meshcat_rigid_body_visualizer import (
    MeshcatRigidBodyVisualizer)

from pydrake.all import (
    DiagramBuilder,
    RigidBodyFrame,
    RigidBodyTree,
    SignalLogger,
    Simulator,
    AddModelInstanceFromUrdfFile,
    FloatingBaseType,
    RbtInverseDynamics,
    VectorSystem,
)

from pydrake.solvers import ik

In [ ]:
class Controller(VectorSystem):
    # konstruktor klasy
    def __init__(self, v):
        # wywolanie konstuktora klasy nadrzednej VectorSystem
        # Podane parametry odpowiadaja opiektowi z 1 portem wyjsciowym i bez portow wejsciowych
        VectorSystem.__init__(self, 0, len(v))
        # ustawienie pola klasy na wartosc poczatkowa 0
        # ta wartosc stanowi w tym przypadku wartosc momentu napedowego
        self.value = v

    # funkcja odpowiadajaca za wyliczanie wartosci wyjsciowej output podczas symulacji
    def _DoCalcVectorOutput(self, context, unused, unused2, output):
        output[:] = self.value

In [ ]:
class Moments():    
    def load_model(self, model):
        self.rbt = RigidBodyTree()
        robot_base_frame = RigidBodyFrame(
                "robot_base_frame", self.rbt.world(),
                [0.0, 0, 0], [0, 0, 0])
        AddModelInstanceFromUrdfFile(model, FloatingBaseType.kFixed,
                                         robot_base_frame, self.rbt)
    
    def calculate_moment(self, state, acceleration):
        builder = DiagramBuilder()
        sp = builder.AddSystem(Controller(state))
        acc_sp = builder.AddSystem(Controller(acceleration))
        inv_dyn = RbtInverseDynamics(self.rbt, RbtInverseDynamics.InverseDynamicsMode.kInverseDynamics)
        invd = builder.AddSystem(inv_dyn)
        logger = builder.AddSystem(SignalLogger(inv_dyn.get_output_port(0).size()))
        logger._DeclarePeriodicPublish(1. / 1000, 0.0)
        builder.Connect(sp.get_output_port(0), invd.get_input_port(0))
        builder.Connect(acc_sp.get_output_port(0), invd.get_input_port(1))
        builder.Connect(invd.get_output_port(0), logger.get_input_port(0))

        diagram = builder.Build()
        simulator = Simulator(diagram)
        simulator.set_target_realtime_rate(-1.0)
        simulator.set_publish_every_time_step(False)

        simulator.StepTo(0)
        return logger.data()[:,0]
  




In [ ]:
#Stworzenie obiektu
m = Moments()

# Załadowanie modelu
m.load_model("wahadlo.urdf")

# Wyznaczenie momentu przy zadanych warosciach zadanych wartosciach polozen i przedosci oraz przyspieszen zlaczowych
# m.calculate_moment([q,dq],[ddq]),
# gdzie q, dq i ddq to dopowiednio: wektor polozen, predkosci i przyspieszen zlaczowych
moments = m.calculate_moment([math.radians(90.0),0],[0])
print(moments)